In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import backend as K
import matplotlib.pyplot as plt
import random

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


In [4]:
x_train=x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test=x_test.reshape(x_test.shape[0], 28, 28, 1)

print(x_train.shape)

input_shape = (28, 28, 1)

(60000, 28, 28, 1)


In [5]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='swish', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=3, activation='swish'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # loss function
              metrics=['accuracy'])

In [8]:
hist = model.fit(x_train, y_train, batch_size=200,
                 epochs=3, validation_data=(x_test, y_test))
print("The model has successfully trained")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/3
300/300 [==============================] - 115s 381ms/step - loss: 0.6746 - accuracy: 0.8040 - val_loss: 0.0836 - val_accuracy: 0.9740
Epoch 2/3
300/300 [==============================] - 114s 381ms/step - loss: 0.0818 - accuracy: 0.9760 - val_loss: 0.0591 - val_accuracy: 0.9810
Epoch 3/3
300/300 [==============================] - 125s 417ms/step - loss: 0.0567 - accuracy: 0.9826 - val_loss: 0.0591 - val_accuracy: 0.9807
The model has successfully trained
Test loss: 0.05914706364274025
Test accuracy: 0.9807000160217285
Saving the model as mnist.h5


GUI Code 

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab
from PIL import Image
import numpy as np


In [ ]:
model = load_model('mnist.h5')

In [ ]:
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    # img = img.reshape(1,28*28)
    img = img.reshape(1,28,28,1)
    img = img/255.0
    print(img)
    #predicting the class
    res = model.predict([img])[0]
    # Number and Accuracy respectively
    return np.argmax(res), max(res)

In [ ]:
class App(tk.Tk):
    def _init_(self):
        tk.Tk._init_(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "black", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect  #(left, top, right, bottom) 
        rect=(a-4,b+4,c+4,d-4)
        print(rect)
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, outline = "white", fill='white', width = 6)
       
app = App()
mainloop()